# Installation
```
git clone git@github.com:vllm-project/llm-compressor.git\
cd llm-compressor\
micromamba create -n weight-analyzer python=3.11\
pip install -e .

```

In [11]:
import transformers
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from llmcompressor.transformers import SparseAutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B"
model = SparseAutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype='auto',
    cache_dir="/nm/drive0/shashata/weight-analysis"
)

model.save_pretrained("/nm/drive0/shashata/weight-analysis/dense_llama_3_8b")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-07-22T13:50:36.491824-0400 | save_pretrained_wrapper | INFO - Inferring a sparsity configuration requires a global sparsity calculation. This can be costly for large models. To skip the calculation of compression statistics set skip_compression_stats=True


Calculating model sparsity: 100%|██████████| 291/291 [00:00<00:00, 1173.48it/s]


# Loading using safetensors for individual weight files

In [25]:
import safetensors

from safetensors import safe_open

tensors = {}
safetensors_path = "/nm/drive0/shashata//weight-analysis/dense_llama_3_8b/model-00001-of-00004.safetensors"
with safe_open(safetensors_path, framework='pt', device='cpu') as f:
    # tensors = safetensors.load(f)
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

for k, v in tensors.items():
    print(k, v.shape)


# Analyzing the weights

In [31]:
# print(tensors.keys())
print(tensors['model.layers.0.mlp.down_proj.weight'])


tensor([[ 0.0087, -0.0151, -0.0090,  ...,  0.0079, -0.0039,  0.0134],
        [ 0.0204, -0.0107, -0.0057,  ...,  0.0010,  0.0172,  0.0011],
        [ 0.0082, -0.0075, -0.0023,  ..., -0.0018,  0.0025, -0.0165],
        ...,
        [ 0.0085, -0.0208,  0.0217,  ..., -0.0199,  0.0081, -0.0129],
        [-0.0135, -0.0059, -0.0110,  ...,  0.0093,  0.0015, -0.0131],
        [-0.0029,  0.0069,  0.0085,  ..., -0.0082, -0.0051, -0.0120]],
       dtype=torch.bfloat16)


In [10]:
model.model.embed_tokens.weight.device

device(type='cuda', index=0)